In [1]:
import cv2
import numpy as np
import tifffile as tiff
import pandas as pd
from ipywidgets import interact, FloatSlider
import pandas as pd

In [2]:
tiff_path = 'X:/RaymondLab/1_Kai/1_cortical_data/3_third_cohort/stage2/466029_f3_stage2_day1.tif'
allen_csv_path = 'allen_mask_no_olf.csv'

In [3]:
# Load the TIFF stack
#tiff_stack = tiff.imread(tiff_path)

In [4]:
# Load the TIFF stack
tiff_stack = tiff.imread(tiff_path)

# Calculate the average projection
avg_projection = np.mean(tiff_stack, axis=0).astype(np.uint16)
max_projection = np.max(tiff_stack, axis=0).astype(np.uint16)
# Load the CSV file containing the cortex regions' coordinates
cortex_data = pd.read_csv(allen_csv_path)
cortex_data

,Unnamed: 0,acronym,name,reference,resolution,label,allen_id,allen_rgb,left_center,right_center,left_x,left_y,right_x,right_y
0,0,FRP,"Frontal pole, cerebral cortex","[540, 570]",10,"{'acronym': 'FRP', 'rgb_triplet': [38, 143, 69...",184,"[38, 143, 69]","[-96.98151950718687, -352.3593429158111]","[96.98151950718693, 787.6406570841889]","[-69.0, -70.0, -71.0, -72.0, -73.0, -74.0, -75...","[-345.5, -345.5, -345.5, -345.5, -345.5, -345....","[69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76....","[-345.5, -345.5, -345.5, -345.5, -345.5, -345...."
1,1,MOp,Primary motor area,"[540, 570]",10,"{'acronym': 'MOp', 'rgb_triplet': [31, 157, 90...",985,"[31, 157, 90]","[-211.0420317794135, -106.62226895665384]","[211.0420317794135, 1033.377731043346]","[-74.0, -75.0, -76.0, -77.0, -78.0, -79.0, -80...","[66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66....","[74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81....","[66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66...."
2,2,MOs,Secondary motor area,"[540, 570]",10,"{'acronym': 'MOs', 'rgb_triplet': [31, 157, 90...",993,"[31, 157, 90]","[-112.3338791933748, -183.8530076888286]","[112.33387919337474, 956.1469923111713]","[-22.0, -23.0, -24.0, -25.0, -26.0, -27.0, -28...","[66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66....","[22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29....","[66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66.5, 66...."
3,3,SSp-n,"Primary somatosensory area, nose","[540, 570]",10,"{'acronym': 'SSp-n', 'rgb_triplet': [24, 128, ...",353,"[24, 128, 100]","[-397.2676367869616, 6.502910360884698]","[397.2676367869616, 1146.5029103608847]","[-447.0, -448.0, -448.5, -448.5, -448.5, -448....","[72.5, 72.5, 72.0, 71.0, 70.0, 69.0, 68.0, 67....","[447.0, 448.0, 448.5, 448.5, 448.5, 448.5, 448...","[72.5, 72.5, 72.0, 71.0, 70.0, 69.0, 68.0, 67...."
4,4,SSp-m,"Primary somatosensory area, mouth","[540, 570]",10,"{'acronym': 'SSp-m', 'rgb_triplet': [24, 128, ...",345,"[24, 128, 100]","[-370.7080974842767, -96.31218553459121]","[370.7080974842768, 1043.6878144654088]","[-444.0, -444.5, -444.5, -444.5, -444.5, -444....","[-14.5, -15.0, -16.0, -17.0, -18.0, -19.0, -20...","[444.0, 444.5, 444.5, 444.5, 444.5, 444.5, 444...","[-14.5, -15.0, -16.0, -17.0, -18.0, -19.0, -20..."
5,5,SSp-un,"Primary somatosensory area, unassigned","[540, 570]",10,"{'acronym': 'SSp-un', 'rgb_triplet': [24, 128,...",182305689,"[24, 128, 100]","[-306.03176567656766, 29.60726072607258]","[306.0317656765676, 1169.6072607260726]","[-275.0, -276.0, -276.5, -276.5, -276.5, -276....","[125.5, 125.5, 125.0, 124.0, 123.0, 122.0, 121...","[275.0, 276.0, 276.5, 276.5, 276.5, 276.5, 276...","[125.5, 125.5, 125.0, 124.0, 123.0, 122.0, 121..."
6,6,PL,Prelimbic area,"[540, 570]",10,"{'acronym': 'PL', 'rgb_triplet': [47, 168, 80]...",972,"[47, 168, 80]","[-28.77936366339577, -317.099910793934]","[28.77936366339577, 822.900089206066]","[-1.0, -1.5, -1.5, -1.5, -1.5, -1.5, -2.0, -2....","[-244.5, -245.0, -246.0, -247.0, -248.0, -249....","[1.0, 1.5, 1.5, 1.5, 1.5, 1.5, 2.0, 2.5, 2.5, ...","[-244.5, -245.0, -246.0, -247.0, -248.0, -249...."
7,7,ACAd,"Anterior cingulate area, dorsal part","[540, 570]",10,"{'acronym': 'ACAd', 'rgb_triplet': [64, 166, 1...",39,"[64, 166, 102]","[-8.913455657492364, -173.20550458715599]","[8.913455657492364, 966.7944954128441]","[-1.0, -2.0, -3.0, -4.0, -4.5, -4.5, -4.5, -4....","[64.5, 64.5, 64.5, 64.5, 64.0, 63.0, 62.0, 61....","[1.0, 2.0, 3.0, 4.0, 4.5, 4.5, 4.5, 4.5, 4.5, ...","[64.5, 64.5, 64.5, 64.5, 64.0, 63.0, 62.0, 61...."
8,8,RSPv,"Retrosplenial area, ventral part","[540, 570]",10,"{'acronym': 'RSPv', 'rgb_triplet': [26, 166, 1...",886,"[26, 166, 152]","[-10.775991425509119, 235.00884244372992]","[10.775991425509119, 1375.00884244373]","[-29.0, -29.5, -29.5, -29.0, -28.5, -28.5, -28...","[352.5, 352.0, 351.0, 350.5, 350.0, 349.0, 348...","[29.0, 29.5, 29.5, 29.0, 28.5, 28.5, 28.0, 27....","[352.5, 352.0, 351.0, 350.5, 350.0, 349.0, 348..."
9,9,RSPd,"Retrosplenial area, dorsal part","[540, 570]",10,"{'acronym': 'RSPd', 'rgb_tr

In [5]:
# Function to calculate the extents and adjust coordinates for centering and scaling
x_offset = 0
y_offset = 0
scale = 100
def calculate_and_adjust_coordinates():
    all_x = []
    all_y = []
    # Gather all coordinates from the dataset to find extents
    for index, row in cortex_data.iterrows():
        all_x.extend(eval(row['left_x']) + eval(row['right_x']))
        all_y.extend(eval(row['left_y']) + eval(row['right_y']))
    
    # Calculate min and max
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)
    width = max_x - min_x
    height = max_y - min_y

    # Determine scaling factor to fit the overlay in the image dimensions
    scale_factor = min(max_projection.shape[0] / height, max_projection.shape[1] / width)

    # Calculate translation to center the overlay
    center_x = (max_projection.shape[1] - (width * scale_factor)) / 2
    center_y = (max_projection.shape[0] - (height * scale_factor)) / 2
    
    return scale_factor, center_x - min_x * scale_factor, center_y - min_y * scale_factor

# Get scale and translation adjustments
scale_factor, translate_x, translate_y = calculate_and_adjust_coordinates()

# Adjust draw_static_boundaries function to include scaling and translation
def draw_adjusted_static_boundaries(scale, tx, ty):
    overlay = np.zeros_like(max_projection, dtype=np.uint8)
    cortex_data['adjusted_left_x'] = None
    cortex_data['adjusted_left_y'] = None
    cortex_data['adjusted_right_x'] = None
    cortex_data['adjusted_right_y'] = None
    for index, row in cortex_data.iterrows():
        left_x = [x * scale + tx for x in eval(row['left_x'])]
        left_y = [y * scale + ty for y in eval(row['left_y'])]
        right_x = [x * scale + tx for x in eval(row['right_x'])]
        right_y = [y * scale + ty for y in eval(row['right_y'])]
        # Update the DataFrame with the adjusted coordinates
        cortex_data.at[index, 'adjusted_left_x'] = left_x
        cortex_data.at[index, 'adjusted_left_y'] = left_y
        cortex_data.at[index, 'adjusted_right_x'] = right_x
        cortex_data.at[index, 'adjusted_right_y'] = right_y
        
        left_coords = np.array(list(zip(left_x, left_y)), dtype=np.int32)
        right_coords = np.array(list(zip(right_x, right_y)), dtype=np.int32)
        
        cv2.polylines(overlay, [left_coords], isClosed=False, color=(255, 255, 0), thickness=1)
        cv2.polylines(overlay, [right_coords], isClosed=False, color=(255, 255, 0), thickness=1)
    
    return overlay

# Create a fixed boundary overlay with adjusted coordinates
boundary_overlay_adjusted = draw_adjusted_static_boundaries(scale_factor, translate_x, translate_y)

# Update the composite image function to use the adjusted overlay
def transform_image_only_adjusted(x_offset, y_offset, scale):
    scale_factor_img = max(scale / 100, 0.01)
    transformed_img = cv2.resize(max_projection.astype(np.float32), None, fx=scale_factor_img, fy=scale_factor_img, interpolation=cv2.INTER_LINEAR)
    transformed_img = cv2.normalize(transformed_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    composite_image = np.zeros_like(max_projection, dtype=np.uint8)
    start_x = int(composite_image.shape[1]/2 - transformed_img.shape[1]/2 + x_offset)
    start_y = int(composite_image.shape[0]/2 - transformed_img.shape[0]/2 + y_offset)
    end_x = start_x + transformed_img.shape[1]
    end_y = start_y + transformed_img.shape[0]
    if start_x < 0 or start_y < 0 or end_x > composite_image.shape[1] or end_y > composite_image.shape[0]:
        sx = max(start_x, 0)
        sy = max(start_y, 0)
        ex = min(end_x, composite_image.shape[1])
        ey = min(end_y, composite_image.shape[0])
        composite_image[sy:ey, sx:ex] = transformed_img[sy-start_y:ey-start_y, sx-start_x:ex-start_x]
    else:
        composite_image[start_y:end_y, start_x:end_x] = transformed_img
    
    mask = boundary_overlay_adjusted > 0
    composite_image[mask] = boundary_overlay_adjusted[mask]
    
    return composite_image

# Update the display function to use the new composite function
def update_image_only_adjusted(val):
    global x_offset, y_offset, scale
    x_offset = cv2.getTrackbarPos('X Offset', 'Registration') - 170
    y_offset = cv2.getTrackbarPos('Y Offset', 'Registration') - 170
    scale = cv2.getTrackbarPos('Scale', 'Registration')
    transformed = transform_image_only_adjusted(x_offset, y_offset, scale)
    cv2.imshow('Registration', transformed)

# Reinitialize OpenCV window with the new function and adjusted overlay
cv2.namedWindow('Registration')
cv2.createTrackbar('X Offset', 'Registration', 170, 340, update_image_only_adjusted)
cv2.createTrackbar('Y Offset', 'Registration', 170, 340, update_image_only_adjusted)
cv2.createTrackbar('Scale', 'Registration', 100, 200, update_image_only_adjusted)

# Display initial image with static, adjusted boundaries
update_image_only_adjusted(0)

# Keep the window open until 'Esc' key is pressed
while True:
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


KeyboardInterrupt: 

In [ ]:
def adjust_tif_stack(stack, scale, x_offset, y_offset):
    # Calculate scaling factor
    scale_factor = max(scale / 100, 0.01)
    adjusted_stack = []
    for frame in stack:
        # Scale each frame
        transformed_frame = cv2.resize(frame.astype(np.uint16), None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
        # Create new blank frame to hold transformed frame
        new_frame = np.zeros_like(frame, dtype=np.uint16)
        start_x = int(new_frame.shape[1]/2 - transformed_frame.shape[1]/2 + x_offset)
        start_y = int(new_frame.shape[0]/2 - transformed_frame.shape[0]/2 + y_offset)
        end_x = start_x + transformed_frame.shape[1]
        end_y = start_y + transformed_frame.shape[0]
        if start_x < 0 or start_y < 0 or end_x > new_frame.shape[1] or end_y > new_frame.shape[0]:
            sx = max(start_x, 0)
            sy = max(start_y, 0)
            ex = min(end_x, new_frame.shape[1])
            ey = min(end_y, new_frame.shape[0])
            new_frame[sy:ey, sx:ex] = transformed_frame[sy-start_y:ey-start_y, sx-start_x:ex-start_x]
        else:
            new_frame[start_y:end_y, start_x:end_x] = transformed_frame
        adjusted_stack.append(new_frame)
    return np.array(adjusted_stack).astype(np.uint16)

adjusted_stack = adjust_tif_stack(tiff_stack, scale, x_offset, y_offset)

In [ ]:
print(y_offset)

In [ ]:
####### FOR DFF STACK ########
def adjust_tif_stack(stack, scale, x_offset, y_offset):
    # Calculate scaling factor
    scale_factor = max(scale / 100, 0.01)
    adjusted_stack = []
    for frame in stack:
        # Scale each frame
        transformed_frame = cv2.resize(frame.astype(np.float32), None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
        # Create new blank frame to hold transformed frame
        new_frame = np.zeros_like(frame, dtype=np.float32)
        start_x = int(new_frame.shape[1]/2 - transformed_frame.shape[1]/2 + x_offset)
        start_y = int(new_frame.shape[0]/2 - transformed_frame.shape[0]/2 + y_offset)
        end_x = start_x + transformed_frame.shape[1]
        end_y = start_y + transformed_frame.shape[0]
        if start_x < 0 or start_y < 0 or end_x > new_frame.shape[1] or end_y > new_frame.shape[0]:
            sx = max(start_x, 0)
            sy = max(start_y, 0)
            ex = min(end_x, new_frame.shape[1])
            ey = min(end_y, new_frame.shape[0])
            new_frame[sy:ey, sx:ex] = transformed_frame[sy-start_y:ey-start_y, sx-start_x:ex-start_x]
        else:
            new_frame[start_y:end_y, start_x:end_x] = transformed_frame
        adjusted_stack.append(new_frame)
    return np.array(adjusted_stack).astype(np.float32)

adjusted_stack = adjust_tif_stack(tiff_stack, scale, x_offset, y_offset)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def display_image_with_overlay(image, cortex_data, region_name, side = 'both'):
    # Filter the row corresponding to the region of interest
    region_row = cortex_data[cortex_data['acronym'] == region_name]
    
    if region_row.empty:
        print(f"No region found with the name {region_name}")
        return
    
    # Extract the coordinates from the filtered row
    left_x = region_row.iloc[0]['adjusted_left_x']
    left_y = region_row.iloc[0]['adjusted_left_y']
    right_x = region_row.iloc[0]['adjusted_right_x']
    right_y = region_row.iloc[0]['adjusted_right_y']
    
    # Convert coordinates to the correct format for plotting
    
    left_coords = np.array(list(zip(left_x, left_y)))
    right_coords = np.array(list(zip(right_x, right_y)))
    
    # Display the image
    plt.imshow(image, cmap='gray')
    
    # Plot the polylines on the image
    if side == 'both':
        plt.plot(left_coords[:, 0], left_coords[:, 1], color='yellow', linewidth=1)
        plt.plot(right_coords[:, 0], right_coords[:, 1], color='yellow', linewidth=1)

    if side == 'left':
        plt.plot(left_coords[:, 0], left_coords[:, 1], color='yellow', linewidth=1)

    if side == 'right':
        plt.plot(right_coords[:, 0], right_coords[:, 1], color='yellow', linewidth=1)
    
    # Show the image with overlay
    plt.title("Image with Overlay")
    plt.axis('off')
    plt.show()


In [ ]:
image = np.std(adjusted_stack, axis=0).astype(np.float32)
region_name_of_interest = "MOs"  # Replace with the actual region name
display_image_with_overlay(image, cortex_data, region_name_of_interest,'right')

In [ ]:
plt.imshow()

In [ ]:
import numpy as np
import cv2

def generate_logical_mask(image_shape, cortex_data, region_name,side='both'):
    # Create a mask with the same dimensions as the image
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    mask2 = np.zeros(image_shape[:2], dtype=np.uint8)

    if region_name == 'all':

        for r in cortex_data['acronym']:
        # Filter the row corresponding to the region of interest
            region_row = cortex_data[cortex_data['acronym'] == r]

            left_x = region_row.iloc[0]['adjusted_left_x']
            left_y = region_row.iloc[0]['adjusted_left_y']
            right_x = region_row.iloc[0]['adjusted_right_x']
            right_y = region_row.iloc[0]['adjusted_right_y']
            
            
            # Convert coordinates to the correct format for polylines
            left_coords = np.array(list(zip(left_x, left_y)), dtype=np.int32)
            right_coords = np.array(list(zip(right_x, right_y)), dtype=np.int32)
            cv2.polylines(mask, [left_coords], isClosed=True, color=1, thickness=2)
            cv2.polylines(mask, [right_coords], isClosed=True, color=1, thickness=2)
            cv2.fillPoly(mask2, [left_coords], color=1)
            cv2.fillPoly(mask2, [right_coords], color=1)
        return ((1-mask)*mask2).astype(bool)
            

    else:
            
        # Filter the row corresponding to the region of interest
        region_row = cortex_data[cortex_data['acronym'] == region_name]
    
        
        if region_row.empty:
            print(f"No region found with the name {region_name}")
            return None
            # Extract the coordinates from the filtered row
        left_x = region_row.iloc[0]['adjusted_left_x']
        left_y = region_row.iloc[0]['adjusted_left_y']
        right_x = region_row.iloc[0]['adjusted_right_x']
        right_y = region_row.iloc[0]['adjusted_right_y']
        
        
        # Convert coordinates to the correct format for polylines
        left_coords = np.array(list(zip(left_x, left_y)), dtype=np.int32)
        right_coords = np.array(list(zip(right_x, right_y)), dtype=np.int32)
    
        # Fill the polygon defined by the left and right coordinates on the mask
        if side == 'both':
            cv2.fillPoly(mask, [left_coords], color=1)
            cv2.fillPoly(mask, [right_coords], color=1)
        if side == 'right':
            cv2.fillPoly(mask, [right_coords], color=1)
        if side == 'left':
            cv2.fillPoly(mask, [left_coords], color=1)
    
        return mask.astype(bool)

def compute_region_trace(stack, cortex_data, region_name,side='both'):
    # Ensure the stack is a 3D array (m, w, h)
    if len(stack.shape) != 3:
        raise ValueError("Stack should be a 3D array of shape (m, w, h)")
    
    # Generate the logical mask for the given region
    mask = generate_logical_mask(stack[0].shape, cortex_data, region_name,side=side)
    
    if mask is None:
        return None
    
    # Compute the mean of the masked pixels for each frame
    trace = [np.mean(frame[mask]) for frame in stack]
    
    return trace

# Example usage:

region_name_of_interest = "all"  # Replace with the actual region name
mask = generate_logical_mask(adjusted_stack.shape[1:3], cortex_data, region_name_of_interest,side='both')
trace = compute_region_trace(adjusted_stack, cortex_data, region_name_of_interest,'right')




In [ ]:
cortex_data.to_csv('/home/lar/Experiments/Daniel/Cortical Analysis/new/240719/cortical_activity/cortical_map.csv')

In [ ]:
mask = generate_logical_mask(adjusted_stack.shape[1:3], cortex_data, 'all',side='both')
trace = compute_region_trace(adjusted_stack, cortex_data, region_name_of_interest,'right')

In [ ]:
tiff.imwrite('/home/lar/Experiments/Daniel/Cortical Analysis/new/240719/cortical_activity/457826M5T_dff_reg.tif', adjusted_stack.astype(np.float16), dtype='float16')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec



# Create a figure with GridSpec
fig = plt.figure(figsize=(25, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 3, 1])

# First subplot: masked mean image
ax1 = plt.subplot(gs[0])
mean_image = np.mean(adjusted_stack, axis=0)
ax1.imshow(mask * mean_image, cmap='gray')
ax1.set_title('Masked Mean Image')

# Second subplot: trace plot spanning two columns
ax2 = plt.subplot(gs[1:])
ax2.plot(trace)
ax2.set_title('Trace Plot')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec

# Assuming adjusted_stack, cortex_data, generate_logical_mask, compute_region_trace are defined

fig = plt.figure(figsize=(25, 30))
gs = gridspec.GridSpec(cortex_data.shape[0], 3, width_ratios=[0.5, 1, 1])

# Create the first plot to set up the shared x-axis
ax_shared = plt.subplot(gs[1:3])
plt.setp(ax_shared.get_xticklabels(), visible=False)
for i in range(1, cortex_data.shape[0] + 1):
    # First subplot: masked mean image
    ax1 = plt.subplot(gs[(i - 1) * 3])
    mean_image = np.mean(adjusted_stack, axis=0)
    mask = generate_logical_mask(adjusted_stack.shape[1:3], cortex_data, cortex_data.iloc[i - 1]['acronym'], side='both')
    ax1.imshow(mask * mean_image, cmap='gray')
    ax1.axis('off')
    ax1.set_title(f"{cortex_data.iloc[i - 1]['acronym']}")
    # Second subplot: trace plot spanning two columns, sharing x-axis
    trace = compute_region_trace(adjusted_stack, cortex_data, cortex_data.iloc[i - 1]['acronym'], 'both')
    
    if i == 1:
        ax2 = plt.subplot(gs[1 + (i - 1) * 3:3 + (i - 1) * 3], sharex=ax_shared)
        ax2.set_title('Trace Plot')  # Title only for the first row
    else:
        ax2 = plt.subplot(gs[1 + (i - 1) * 3:3 + (i - 1) * 3], sharex=ax_shared)
    
    
    ax2.plot(trace)
    ax2.set_xlim([0, len(trace)])
    ax2.set_ylim([np.min(trace), np.max(trace)])
    
    # Hide x-axis ticks for all but the last plot
    if i < cortex_data.shape[0]:
        plt.setp(ax2.get_xticklabels(), visible=False)
    else:
        plt.setp(ax2.get_xticklabels(), visible=True)
    
    # Make layout tight for each subplot
    plt.tight_layout()

# Adjust overall layout
plt.tight_layout()

plt.show()


In [ ]:
data = {'cortex':adjusted_stack, 'coordinates': cortex_data}

In [ ]:
import pickle

data = {'cortex':adjusted_stack, 'coordinates': cortex_data}

with open('my_dict.pkl', 'wb') as file:
    pickle.dump(a, file)

In [ ]:
with open('my_dict.pkl', 'rb') as file:
    loaded_dict = pickle.load(file)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

def calculate_center(x_coords, y_coords):
    center_x = np.mean(x_coords)
    center_y = np.mean(y_coords)
    return center_x, center_y

def plot_image_with_regions_and_legend(image, cortex_data):
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.imshow(image, cmap='gray')
    
    texts = []
    acronyms = []
    full_names = []
    
    for index, row in cortex_data.iterrows():
        left_x = row['adjusted_left_x']
        left_y = row['adjusted_left_y']
        right_x = row['adjusted_right_x']
        right_y = row['adjusted_right_y']
        
        # Calculate center of the region
        all_x = left_x + right_x
        all_y = left_y + right_y
        l_center_x, l_center_y = calculate_center(left_x, left_y)
        r_center_x, r_center_y = calculate_center(right_x, right_y)
        center_x, center_y = calculate_center(all_x, all_y)
        
        # Plot the center dot
        ax.plot(l_center_x, l_center_y, 'ro')  # Red dot
        ax.plot(r_center_x, r_center_y, 'go')  # Red dot
        ax.axis('off')
        # Collect text objects for adjustment
        text = ax.text(l_center_x, l_center_y, row['acronym'], color='white', fontsize=12,
                       bbox=dict(facecolor='red', alpha=0.5, boxstyle='round,pad=0.5'))
        texts.append(text)
        text = ax.text(r_center_x, r_center_y, row['acronym'], color='white', fontsize=12,
                       bbox=dict(facecolor='green', alpha=0.5, boxstyle='round,pad=0.5'))
        texts.append(text)
        
        # Collect acronyms and full names
        acronyms.append(row['acronym'])
        full_names.append(row['name'])
    
    # Adjust text annotations to avoid overlap
    adjust_text(texts, only_move={'points': 'y', 'texts': 'y'}, arrowprops=dict(arrowstyle="->", color='r', lw=0.5))
    
    # Add the legend on the right side
    ax_leg = fig.add_axes([0.85, 0.1, 0.1, 0.8], frame_on=False)
    ax_leg.axis('off')
    
    for i, (acronym, full_name) in enumerate(zip(acronyms, full_names)):
        y_pos = 1 - (i / len(acronyms)) * 1.1
        # Display acronym with box
        acronym_text = ax_leg.text(0, y_pos, f'{acronym}', color='white', fontsize=10,
                                   bbox=dict(facecolor='red', alpha=0.8, boxstyle='round,pad=0.5'))
        # Calculate the width of the acronym box
        bbox = acronym_text.get_window_extent(renderer=fig.canvas.get_renderer())
        width = bbox.width / fig.dpi  # Convert from pixels to inches
        # Display full name next to the box with offset
        ax_leg.text(width+0.05, y_pos, f'{full_name}', color='black', fontsize=10, verticalalignment='center')
    
    plt.show()

# Example usage:
# Assuming 'image' is your image array and 'cortex_data' is your DataFrame
plot_image_with_regions_and_legend(image, cortex_data)


In [ ]:
!pip install tdt

In [ ]:
import tdt

data = tdt.read_block('717M5/717M5_May28_2')

In [ ]:
data

In [ ]:
Fs = data.streams._465G['fs']

In [ ]:
data.streams

In [ ]:
data.epocs.Tal_['onset'].shape

In [ ]:
data.streams._465G['data'].shape

In [ ]:
from scipy.signal import detrend
from scipy.stats import zscore
start = int(data.epocs.rec_['onset']*Fs +Fs)
stop = int(data.epocs.rec_['offset']*Fs - Fs)
signal_G = data.streams._465G['data'][start:stop]
signal_R = data.streams._560R['data'][start:stop]
dff_signal_G = detrend((signal_G - np.mean(signal_G))/np.mean(signal_G))*100
dff_signal_R = detrend((signal_R - np.mean(signal_R))/np.mean(signal_R))*100
l = len(signal_G)

ts = np.linspace(0,l/Fs,l)

plt.plot(ts,dff_signal_G,c='darkgreen')
plt.plot(ts,dff_signal_R,c='darkred')

In [ ]:
t_trials_on = [int(x - start -1) for x in data.epocs.Tal_['onset']*Fs]
t_trials_off = [int(x - start -1) for x in data.epocs.Tal_['offset']*Fs]

In [ ]:
trial_signal_G = []
trial_signal_R = []
for i in t_trials_on:
    trial_signal_G.append(dff_signal_G[i-2*int(Fs):i+3*int(Fs)])
    trial_signal_R.append(dff_signal_R[i-2*int(Fs):i+3*int(Fs)])

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(2, hspace=0)
axs = gs.subplots(sharex=True, sharey=True)
fig.suptitle('Green and Red Photometry Signal')

for trace in trial_signal_G:
    axs[0].plot(ts, zscore(trace), color='darkgreen', alpha=0.01)
for trace in trial_signal_R:
    axs[1].plot(ts, zscore(trace), color='darkred', alpha=0.01)


axs[0].plot(ts,zscore(np.mean(trial_signal_G,axis=0)),c='darkgreen',lw=3)
axs[1].plot(ts,zscore(np.mean(trial_signal_R,axis=0)),c='darkred',lw=3)

axs[0].set_ylabel('GluSnFR z-score')
axs[1].set_ylabel('PreSyn Ca z-score')

axs[0].axvline(0,c='black',ls='--')
axs[1].axvline(0,c='black',ls='--')
plt.ylim(-2.5,3)
plt.xlim(-2,3)
axs[0].grid('on')
axs[1].grid('on')


# Hide x labels and tick labels for all but bottom plot.
for ax in axs:
    ax.label_outer()

In [ ]:
import pickle

data = {'cortex':adjusted_stack, 'striatum': {'Green':dff_signal_G, 'Red':dff_signal_R, 'trial_on': t_trials_on, 'trial_off': t_trials_off},'coordinates': cortex_data}

with open('my_dict.pkl', 'wb') as file:
    pickle.dump(data, file)